In [1]:
import os
import typing as tp
import pandas as pd
import sentence_transformers

/opt/miniconda3/envs/ml/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
W0625 18:41:02.429000 55653 site-packages/torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


In [2]:
model_id: str = "sentence-transformers/all-MiniLM-L6-v2"
dataset_path: str = "/Users/chrnegor/Documents/study/UrTraining/ml/scripts/data/dataset.csv"

In [3]:
df = pd.read_csv(dataset_path)
train_df = df.sample(frac=0.8)
test_df = df.drop(train_df.index)

In [4]:
training_data = []

for idx, row in train_df.iterrows():
    query: str = row['user_profile_w_meta']
    positive: str = row['course_description']
    negatives: tp.List[str] = []
    while len(negatives) < 4:
        negative_item = df.sample(1)
        if abs(int(negative_item['course_id']) - int(row['course_id'])) > 3:
            negatives.append(negative_item['course_description'])
    
    for negative in negatives:
        training_data.append({
            'query': query,
            'positive': positive,
            'negative': negative
        })

/var/folders/8r/zrjm4hxd2752djqdmtnnvk700000gn/T/ipykernel_55653/3085943356.py:9: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  if abs(int(negative_item['course_id']) - int(row['course_id'])) > 3:


In [5]:
training_data = pd.DataFrame(training_data, columns=['query', 'positive', 'negative'])
training_data.to_csv("data/training_data.csv", index=False)

---

#### Training

In [6]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

model_id: str = "sentence-transformers/all-MiniLM-L6-v2"
dataset_path: str = "data/training_data.csv"

df = pd.read_csv(dataset_path)

train_examples = [
    InputExample(texts=[row['query'], row['positive'], row['negative']])
    for _, row in df.iterrows()
]
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
model = SentenceTransformer('all-MiniLM-L6-v2')
train_loss = losses.TripletLoss(model=model)

In [7]:
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=1,
    warmup_steps=10,
    show_progress_bar=True
)

TypeError: Accelerator.unwrap_model() got an unexpected keyword argument 'keep_torch_compile'